# Parsing Text

In [ ]:

#!pip install PyPDF2
#!pip install --upgrade pymupdf
#!pip install ebooklib

In [2]:
!pip install PyPDF2

     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     ------ ------------------------------ 41.0/232.6 kB 960.0 kB/s eta 0:00:01
     ------------- ------------------------- 81.9/232.6 kB 1.1 MB/s eta 0:00:01
     ------------------------------------ - 225.3/232.6 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 232.6/232.6 kB 1.8 MB/s eta 0:00:00


# Read Epub

In [3]:
!pip install --upgrade pymupdf

  Obtaining dependency information for pymupdf from https://files.pythonhosted.org/packages/1f/91/1203464adde5d941993a3f919c34ae7921dc330a27dc336c56a2f948e829/PyMuPDF-1.23.8-cp38-none-win_amd64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.23.7 from https://files.pythonhosted.org/packages/01/24/99cfd4e23d42c2f8032a1b55353193c7ca7fb43915432e035afd5e7c52ec/PyMuPDFb-1.23.7-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
    --------------------------------------- 0.1/3.5 MB 1.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/3.5 MB 3.0 MB/s eta 0:00:02
   ---- ----------------------------------- 0.4/3.5 MB 3.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.5/3.5 MB 2.8 MB/s eta 0:00:02
   ------- -------------------------------- 0.6/3.5 MB 2.8 MB/s eta 0:00:02
   ------- -------------------------------- 0.7/3.5 MB 2.7 MB/s eta 0:00:02
   --------- ------------------------------ 0.8/3.5 

In [4]:
!pip install ebooklib

     ---------------------------------------- 0.0/115.5 kB ? eta -:--:--
     -------------------- ------------------ 61.4/115.5 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 115.5/115.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38801 sha256=c225435424c2e1e44161786f9e9cc70b6144710e7f23bcf41b42854a15892dc8
  Stored in directory: c:\users\wutai\appdata\local\pip\cache\wheels\6d\c5\f9\c615f68c24f539a89184ea938cd630d5973d582af6ced69140
Successfully built ebooklib


In [88]:
import ebooklib
from ebooklib import epub
import re
import os

In [89]:
def read_epub_book(epub_file, ID):
    book = epub.read_epub(epub_file)
    text = "" 

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            content = re.sub("&#13;", ' ', content)
            text += content
            #paragraphs.extend(content.strip().split("&#13;"))
    book_item = {'text': text, 'bookID':ID}
    return book_item

In [24]:
read_epub_book('The Little Prince.epub',1)

{'text': ' THE LITTLE PRINCE Antoine de Saint-Exupéry MACMILLAN  CONTENTS CHAPTER 1 CHAPTER 2 CHAPTER 3 CHAPTER 4 CHAPTER 5 CHAPTER 6 CHAPTER 7 CHAPTER 8 CHAPTER 9 CHAPTER 10 CHAPTER 11 CHAPTER 12 CHAPTER 13 CHAPTER 14 CHAPTER 15 CHAPTER 16 CHAPTER 17 CHAPTER 18 CHAPTER 19 CHAPTER 20 CHAPTER 21 CHAPTER 22 CHAPTER 23 CHAPTER 24 CHAPTER 25 CHAPTER 26 CHAPTER 27  CHAPTER 1 I was six years old when I spotted a magnificent picture in a book about an ancient forest. The picture showed a boa constrictor in the act of swallowing a wild beast. This is what it looked like: It said in the book: ‘A boa constrictor swallows its prey whole, without chewing it. After this, it is unable to move and sleeps through the next six months – this is needed for digestion.’ I thought deeply about the experiences of the forest. Then, with care and a coloured pencil, I succeeded in making my first drawing. Drawing Number One looked like this: I showed my fine work to the grown-ups, and asked if the drawing scare

# Read epub into paragraphs of max length

In [90]:
def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []

    for s in strings:
        if len(merged_string) <= min_length:# and len(s)<max_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
            splitParagraph = merged_string.split('.')
            splitParagraphRePoint = []
            for sp in splitParagraph:
                splitParagraphRePoint.append(sp+'.')
            merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
            merged_strings.extend(merged)
            merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s

    if merged_string:
        merged_strings.append(merged_string)

    return merged_strings

In [91]:
def read_epub_paragraphs(epub_file, ID):
    book = epub.read_epub(epub_file)
    paragraphs = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            paragraphs.extend(content.strip().split("&#13;"))
    paragraphs = merge_strings_until_limit(paragraphs, 200, 1000)
    paragraphs = [{'paragraph':paragraphs[i], 'nr':i,'bookID':ID} for i in range(len(paragraphs)) if len(paragraphs[i])>50]
    return paragraphs[1:-1]

In [27]:
read_epub_paragraphs('The Little Prince.epub',1)

[{'paragraph': 'CHAPTER 1 I was six years old when I spotted a magnificent picture in a book about an ancient forest. The picture showed a boa constrictor in the act of swallowing a wild beast. This is what it looked like: It said in the book: ‘A boa constrictor swallows its prey whole, without chewing it.',
  'nr': 1,
  'bookID': 1},
 {'paragraph': ' After this, it is unable to move and sleeps through the next six months – this is needed for digestion.’ I thought deeply about the experiences of the forest. Then, with care and a coloured pencil, I succeeded in making my first drawing.',
  'nr': 2,
  'bookID': 1},
 {'paragraph': ' Drawing Number One looked like this: I showed my fine work to the grown-ups, and asked if the drawing scared them. ‘Scared? Why, it is only a hat!’ But it was not a hat! It was very clearly a boa constrictor digesting an elephant.',
  'nr': 3,
  'bookID': 1},
 {'paragraph': ' But the grown-ups need explanations for everything. So, I made another drawing showin

# Vectorise Text - 1
First find a way to check if a word is valid

In [ ]:
#!pip install pyenchant

# you'll also need to install the c library: 
# check: 
# https://pyenchant.github.io/pyenchant/install.html
# https://stackoverflow.com/questions/34026324/cant-install-pyenchant-on-windows-even-after-i-installed-enchant-manually

In [18]:
!pip install pyenchant

In [92]:
import os
import enchant

In [93]:
ENGLISH_DICT1 = enchant.Dict("en_UK")
ENGLISH_DICT2 = enchant.Dict("en_US")

In [94]:
def is_english_word(word):
    # Initialize the Enchant English dictionary
    return (ENGLISH_DICT1.check(word) or ENGLISH_DICT2.check(word))

In [95]:
is_english_word('test')

True

In [32]:
#if you can't get enchant to work, an alternative is:

In [96]:
import nltk

# Download the 'words' corpus
nltk.download('words')

# Now, check if "test" is in the words corpus
from nltk.corpus import words
print("test" in words.words())

True


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\wutai\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [97]:
from nltk.corpus import words
"test" in words.words()

True

# Then we can start vectorising
First step is to preprocess the paragraphs

In [8]:
!pip install gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/3e/b7/fba98a65efea29a7d8bf25ade2db67e34ebab8e63769e8927d0a4d42a84f/gensim-4.3.2-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 2.3 MB/s eta 0:00:11
   ---------------------------------------- 0.2/24.0 MB 3.0 MB/s eta 0:00:08
    --------------------------------------- 0.5/24.0 MB 3.5 MB/s eta 0:00:07
   - -------------------------------------- 0.7/24.0 MB 4.2 MB/s eta 0:00:06
   - -------------------------------------- 1.0/24.0 MB 4.7 MB/s eta 0:00:05
   -- ------------------------------------- 1.4/24.0 MB 5.1 MB/s eta 0:00:05
   -- ------------------------------------- 1.6/24.0

In [98]:
import gensim
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, stopwords, names

In [99]:
paragraphs = read_epub_paragraphs('The Little Prince.epub',1)

In [100]:
paragraph = paragraphs[25]['paragraph']

In [101]:
paragraph

' I expect others to take my problems seriously. Then he said, ‘So you have come from the sky too! From which planet?’ At that instant I realized he was giving me an important hint about himself, and I said, ‘You’re from a different planet, aren’t you?’ He did not respond.'

In [102]:
words = gensim.utils.simple_preprocess(paragraph, min_len = 3, deacc=True)
print(words)

['expect', 'others', 'take', 'problems', 'seriously', 'then', 'said', 'you', 'have', 'come', 'from', 'the', 'sky', 'too', 'from', 'which', 'planet', 'that', 'instant', 'realized', 'was', 'giving', 'important', 'hint', 'about', 'himself', 'and', 'said', 'you', 'from', 'different', 'planet', 'aren', 'you', 'did', 'not', 'respond']


In [103]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
print(lemmatized_words)

['expect', 'others', 'take', 'problem', 'seriously', 'then', 'said', 'you', 'have', 'come', 'from', 'the', 'sky', 'too', 'from', 'which', 'planet', 'that', 'instant', 'realized', 'wa', 'giving', 'important', 'hint', 'about', 'himself', 'and', 'said', 'you', 'from', 'different', 'planet', 'aren', 'you', 'did', 'not', 'respond']


In [104]:
 stop_words = stopwords.words("english")

In [105]:
filtered_words = [word for word in lemmatized_words if ((word not in stop_words)and(is_english_word(word)))]
print(filtered_words)

['expect', 'others', 'take', 'problem', 'seriously', 'said', 'come', 'sky', 'planet', 'instant', 'realized', 'giving', 'important', 'hint', 'said', 'different', 'planet', 'respond']


In [106]:
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in filtered_words]
print(stemmed_words)

['expect', 'other', 'take', 'problem', 'serious', 'said', 'come', 'sky', 'planet', 'instant', 'realiz', 'give', 'import', 'hint', 'said', 'differ', 'planet', 'respond']


In [107]:
processed_doc = " ".join(stemmed_words)
print(processed_doc)

expect other take problem serious said come sky planet instant realiz give import hint said differ planet respond


# For homework, combine the above to a function that can process a list of paragraphs and return a list of processed paragraphs (hint: only assign the stop words, lemmatizer and stemmer once)

In [108]:
paragraphs = read_epub_paragraphs("The Little Prince.epub",1)

In [109]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(paragraphs):
    processed_paragraphs = []

    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)

        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

        filtered_words = [word for word in lemmatized_words if (word not in stop_words) and (is_english_word(word))]

        stemmed_words = [stemmer.stem(word) for word in filtered_words]

        processed_paragraph = " ".join(stemmed_words)
        processed_paragraphs.append(processed_paragraph)

    return processed_paragraphs

In [62]:
processedDoc = preprocess([p['paragraph'] for p in paragraphs])
print(processedDoc)

['chapter six year old spot magnific pictur book ancient forest pictur show boa constrictor act swallow wild beast look like said book boa constrictor swallow prey whole without chew', 'unabl move sleep next six month need digest thought deepli experi forest care colour pencil succeed make first draw', 'draw number one look like show fine work grown up ask draw scare scare hat hat clearli boa constrictor digest eleph', 'grown up need explan everyth made anoth draw show eleph insid boa constrictor draw number two look like time grown up advis put away represent boa constrictor insid outsid instead spend time learn geographi histori arithmet grammar', 'six year age gave promis career artist disappoint failur draw number one draw number two grown up find hard understand anyth tire child alway explain thing', 'henc chose differ occup learn fli airplan flew world found geographi use easili distinguish china knowledg import get lost dark', 'year encount mani peopl import spent great deal tim

##### you should get something like this:

In [63]:
processedDoc

['chapter six year old spot magnific pictur book ancient forest pictur show boa constrictor act swallow wild beast look like said book boa constrictor swallow prey whole without chew',
 'unabl move sleep next six month need digest thought deepli experi forest care colour pencil succeed make first draw',
 'draw number one look like show fine work grown up ask draw scare scare hat hat clearli boa constrictor digest eleph',
 'grown up need explan everyth made anoth draw show eleph insid boa constrictor draw number two look like time grown up advis put away represent boa constrictor insid outsid instead spend time learn geographi histori arithmet grammar',
 'six year age gave promis career artist disappoint failur draw number one draw number two grown up find hard understand anyth tire child alway explain thing',
 'henc chose differ occup learn fli airplan flew world found geographi use easili distinguish china knowledg import get lost dark',
 'year encount mani peopl import spent great de

### Now turn these into vectors

In [52]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/52/2d/ad6928a578c78bb0e44e34a5a922818b14c56716b81d145924f1f291416f/scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
    --------------------------------------- 0.1/9.3 MB 2.9 MB/s eta 0:00:04
   - -------------------------------------- 0.3/9.3 MB 3.1 MB/s eta 0:00:03
   - -------------------------------------- 0.5/9.3 MB 2.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/9.3 MB 3.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/9.3 MB 3.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/9.3 MB 2.9 MB/s eta 0:00:03
   ---- ---------------------------------

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [111]:
vectorizer = TfidfVectorizer(min_df=3)

In [112]:
# look up and study TF-IDF in your own time

In [113]:
tfidf_matrix = vectorizer.fit_transform(processedDoc)

In [114]:
tfidf_matrix

<317x494 sparse matrix of type '<class 'numpy.float64'>'
	with 4522 stored elements in Compressed Sparse Row format>

In [115]:
len(processedDoc)

317

In [116]:
vectorizer.get_feature_names_out()

array(['abl', 'absolut', 'across', 'activ', 'ad', 'admir', 'afraid',
       'ago', 'air', 'alon', 'along', 'alreadi', 'also', 'alway', 'among',
       'amus', 'ancient', 'anoth', 'answer', 'anymor', 'anyon', 'anyth',
       'anywher', 'appear', 'arm', 'around', 'arriv', 'ask', 'asteroid',
       'astronom', 'author', 'away', 'back', 'bad', 'baobab', 'beauti',
       'becom', 'began', 'begin', 'believ', 'bell', 'belong', 'besid',
       'best', 'big', 'bird', 'bit', 'boa', 'bodi', 'bolt', 'book',
       'born', 'box', 'boy', 'breath', 'bucket', 'bush', 'busi',
       'businessman', 'butterfli', 'came', 'cannot', 'care', 'carri',
       'case', 'caterpillar', 'caus', 'certainli', 'chair', 'chang',
       'chapter', 'chicken', 'child', 'civil', 'clean', 'clear',
       'clearli', 'close', 'cold', 'colour', 'come', 'command', 'complet',
       'complic', 'concern', 'confus', 'constrictor', 'continu', 'cough',
       'could', 'count', 'cours', 'cri', 'danger', 'day', 'deal', 'death',
      

In [117]:
len(vectorizer.get_feature_names_out())

494

In [118]:
vectorizer = TfidfVectorizer(min_df=3)
tfidf_matrix = vectorizer.fit_transform(processedDoc)

### Now we can search
##### The purpose of this section of code is to calculate the similarity between a new paragraph (query) and an existing paragraph using the TF-IDF matrix.

In [119]:
from sklearn.metrics.pairwise import cosine_similarity

In [120]:
# Vectorise a new paragraph:
query = 'temples and the moon with a man'
processedQuery = preprocess([query])[0]
print(processedQuery)

templ moon man


In [121]:
# should give:
processedQuery = 'templ moon man'

In [122]:
query_vector = vectorizer.transform([processedQuery])

In [123]:
query_vector

<1x494 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [124]:
similarities = cosine_similarity(query_vector, tfidf_matrix)

In [125]:
similarities

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.40590959, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.19584016, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.14652675, 0.22532208, 0.11853824, 0.  

In [126]:
len(similarities[0])

317

In [127]:
# this function argmax() gets the index with the highes value in the list. you can also get a sorted list if 
# indices. In the homework use this to get more than one option for similarity. 

nearest_neighbor_index = similarities.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities[0, nearest_neighbor_index]

In [128]:
nearest_neighbor_index, similarity_score

(8, 0.405909592102243)

In [150]:
most_similar_index = similarities.argmax()
most_similar_score = similarities[0, most_similar_index]
most_similar_paragraph = paragraphs[most_similar_index]

print(f"Most similar paragraph (Index {most_similar_index}) with similarity score {most_similar_score}:\n")
print(most_similar_paragraph)


Most similar paragraph (Index 8) with similarity score 0.405909592102243:

{'paragraph': ' Oh how it pleased the grown-ups to have met such a logical man..', 'nr': 9, 'bookID': 1}


In [129]:
#paragraphs[300]

{'paragraph': '’ I said nothing. ‘It will really be delightful! You will have five hundred million little bells, and I will have five hundred million wells.’ Then he fell silent, because he had started to weep. * ‘Here we are.',
 'nr': 306,
 'bookID': 1}

## This works quite well, but with a lot of data it becomes too heavy, so we apply SVD for dimensionality reduction

In [130]:
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD

In [131]:
# Perform Truncated SVD for dimensionality reduction
n_components = 250  # Number of components to keep after reduction
svd = TruncatedSVD(n_components=n_components, algorithm='randomized')
reduced_matrix = svd.fit_transform(tfidf_matrix)

# The reduced_matrix now contains the dimensionality-reduced data

# If you want to project new data into the reduced space, you can use:
#new_data = ...  # Your new sparse matrix data
#reduced_new_data = svd.transform(new_data)

In [132]:
reduced_query_vec = svd.transform(query_vector)

In [133]:
len(reduced_query_vec[0])

250

In [134]:
similarities2 = cosine_similarity(reduced_query_vec, reduced_matrix)

In [135]:
nearest_neighbor_index = similarities2.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities2[0, nearest_neighbor_index]

In [136]:
nearest_neighbor_index

8

In [152]:
nearest_neighbor_index = similarities.argmax()
similarity_score = similarities[0, nearest_neighbor_index]

most_similar_paragraph = paragraphs[nearest_neighbor_index]

print("Most Similar Paragraph:")
print("Paragraph Index:", nearest_neighbor_index)
print("Similarity Score:", similarity_score)
print(most_similar_paragraph)


Most Similar Paragraph:
Paragraph Index: 8
Similarity Score: 0.405909592102243
{'paragraph': ' Oh how it pleased the grown-ups to have met such a logical man..', 'nr': 9, 'bookID': 1}


In [137]:
paragraphs[280]

{'paragraph': ' I reached the wall just in time to catch my prince; his face was so very pale. ‘What is going on?’ I demanded. ‘You’ve started talking to snakes?’ I loosened the golden muffler that he always had on. I rubbed his temples, and gave him some water to drink.',
 'nr': 286,
 'bookID': 1}

# Vectorise Text - 2

In [138]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# https://radimrehurek.com/gensim/models/doc2vec.html

#### Define another preprocess function that return lemmatized (not stemmed) filtered words in a list, like this:

In [140]:
def preprocess2(paragraphs):
    processed_texts = []

    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)

        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

        filtered_words = [word for word in lemmatized_words if (word not in stop_words) and (is_english_word(word))]

        processed_texts.append(filtered_words)

    return processed_texts



# print first
print(texts[0])


['chapter', 'six', 'year', 'old', 'spotted', 'magnificent', 'picture', 'book', 'ancient', 'forest', 'picture', 'showed', 'boa', 'constrictor', 'act', 'swallowing', 'wild', 'beast', 'looked', 'like', 'said', 'book', 'boa', 'constrictor', 'swallow', 'prey', 'whole', 'without', 'chewing']


In [ ]:
# make the list contain words

In [79]:
texts = preprocess2([p['paragraph'] for p in paragraphs])

In [80]:
texts[0]

['chapter',
 'six',
 'year',
 'old',
 'spotted',
 'magnificent',
 'picture',
 'book',
 'ancient',
 'forest',
 'picture',
 'showed',
 'boa',
 'constrictor',
 'act',
 'swallowing',
 'wild',
 'beast',
 'looked',
 'like',
 'said',
 'book',
 'boa',
 'constrictor',
 'swallow',
 'prey',
 'whole',
 'without',
 'chewing']

In [76]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
model = Doc2Vec(vector_size=250, min_count=0, alpha = 0.025, min_alpha=0.025, epochs=100)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [77]:
test = model.predict_output_word(['temple', 'lives', 'man', 'moon'])
print(test)

[('man', 0.44837084), ('last', 0.005990436), ('wise', 0.0058239256), ('continued', 0.005623949), ('red', 0.00508381), ('balloon', 0.00451276), ('job', 0.0043986714), ('faced', 0.0040926374), ('exactly', 0.003952945), ('discovered', 0.0038744677)]


In [81]:
test

[('man', 0.44837084),
 ('last', 0.005990436),
 ('wise', 0.0058239256),
 ('continued', 0.005623949),
 ('red', 0.00508381),
 ('balloon', 0.00451276),
 ('job', 0.0043986714),
 ('faced', 0.0040926374),
 ('exactly', 0.003952945),
 ('discovered', 0.0038744677)]

In [82]:
query = 'a man thinking about his life under the full moon'
tokens = preprocess2([query])[0]
vector = model.infer_vector(tokens)

In [83]:
tokens

['man', 'thinking', 'life', 'full', 'moon']

In [84]:
vector

array([ 0.15658076, -1.3394378 ,  0.12198712,  0.0286135 ,  0.6261842 ,
        0.03913956,  0.4091876 , -0.21594837, -0.18994251,  0.09017015,
        0.39318904, -0.7542466 ,  0.75538015,  0.21598528, -0.6672879 ,
        0.12838556,  0.446439  , -0.17235377,  0.11057699, -0.76015955,
        0.58974737, -0.6991324 ,  0.2278641 , -0.19352864, -0.09951264,
        0.5613363 ,  0.22140448,  0.1414102 ,  0.93932253, -0.2371111 ,
        0.08287571,  0.6335135 ,  0.5116496 , -0.85115176, -0.02600677,
       -0.5341877 ,  0.1154904 ,  0.3589679 ,  0.5247257 ,  0.19901419,
       -0.11824181,  0.12351158, -0.10455567,  0.12019394, -0.29756206,
       -0.0906736 , -0.8716318 ,  0.55769736,  0.47122973,  0.54805154,
        0.17015071,  0.11610024, -0.24937679,  0.6275997 , -0.5260427 ,
        0.10475048,  0.11589684, -0.40188432, -0.32136226,  0.06230603,
       -0.600245  ,  0.00223175, -0.31271988,  0.14890282,  0.5286565 ,
        0.07943494, -0.15850714, -0.43423006, -0.460831  , -0.07

In [85]:
model.dv.most_similar(positive=[vector])

[(82, 0.33492717146873474),
 (15, 0.31450653076171875),
 (134, 0.22865022718906403),
 (10, 0.22234073281288147),
 (170, 0.21003201603889465),
 (153, 0.20427441596984863),
 (133, 0.2022194117307663),
 (243, 0.19281361997127533),
 (77, 0.186676025390625),
 (122, 0.17858120799064636)]

In [149]:
most_similar_doc = model.dv.most_similar(positive=[vector])[0]
index_of_most_similar = most_similar_doc[0]
similarity_score = most_similar_doc[1]
most_similar_paragraph = paragraphs[index_of_most_similar]

print(f"Most similar paragraph (Index {index_of_most_similar}) with similarity score {similarity_score}:\n")
print(most_similar_paragraph)


Most similar paragraph (Index 82) with similarity score 0.33492717146873474:

{'paragraph': ' Forgive me, my petals are such a mess.’ But the little prince was full of admiration: ‘Oh! How beautiful you are!’ ‘Am I not?’ the flower responded sweetly. ‘And I was born along with the sun.’ The little prince could tell that she wasn’t exactly humble.', 'nr': 85, 'bookID': 1}


In [86]:
#paragraphs[212]

{'paragraph': ' ‘I am not tame.’ ‘Oh! I’m sorry,’ said the little prince. But, after some thought, he added: ‘What does “tame” mean?’ ‘You don’t live here,’ said the fox. ‘What are you searching for?’ ‘I am searching for people,’ said the little prince.',
 'nr': 218,
 'bookID': 1}

# Text and Images

## PDF

In [141]:
import PyPDF2
import fitz  # PyMuPDF

In [142]:
pdf_file_path = "Algorithmic Problem Complexity.pdf"
pdf_document = fitz.open(pdf_file_path)
page = pdf_document.load_page(9)
textInfo = page.get_text('blocks',flags=1+2+8)

In [143]:
text = [t[4] for t in textInfo]

In [144]:
text

[' \n10 \n',
 'partial membership) function. An indication function is equal to one when the object \n',
 'satisfies the conditions of the problem and to zero when an object does not satisfy these \n',
 'conditions. A partial indication function is equal to one when the object satisfies the \n',
 'conditions of the problem and is undefined otherwise. Having an indication function \n',
 'f(x), we can find a necessary x by computing values of f(x). This is a construction \n',
 'problem. The object a for which f(a) = 1 gives us a solution to the initial problem. \n',
 'b) Selection problem reduction:  \n',
 'If we have a selection problem Q, then we can change it to the following \n',
 'construction problem: “Select what you need and then build a copy.” Another way of \n',
 'reduction is, as in the previous case, to build an indication function or a partial \n',
 'indication function and compute its values. The value 1 will indicate the object we \n',
 'need. \n',
 'c) Test problem reduct

In [145]:
def extract_text_from_pdf(pdf_path):
    text = {}
    pdf_document = fitz.open(pdf_path)
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        textInfo = page.get_text("blocks", flags=1+2+8)
        paragraphs = [re.sub('\n', ' ', t[4]) for t in textInfo]
        text[page_num] = paragraphs
    pdf_document.close()
    return text

def extract_images_from_pdf(pdf_path, image_output_dir):
    pdf_document = fitz.open(pdf_path)
    images = []

    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        image_list = page.get_images(full=True)

        for img_index, img_info in enumerate(image_list):
            image = pdf_document.extract_image(img_info[0])
            image_path = f"{image_output_dir}/page_{page_num + 1}_image_{img_index}.png"
            with open(image_path, "wb") as image_file:
                image_file.write(image["image"])

            images.append(image_path)

    pdf_document.close()
    return images

In [146]:
pdf_file_path = "Algorithmic Problem Complexity.pdf"
output_text = extract_text_from_pdf(pdf_file_path)

In [147]:
output_text

{0: ['  1  ',
  'Algorithmic Problem Complexity   ',
  '  Mark Burgin   ',
  'Department of Computer Science  University of California, Los Angeles  405 Hilgard Ave.  Los Angeles, CA 90095    ',
  'Universes of virtually unlimited  complexity can be created in the  form of computer programs.  Joseph Weizenbaum    ',
  'Abstract: People solve different problems and know that some of them are simple,  ',
  'some are complex and some insoluble. The main goal of this work is to develop a  ',
  'mathematical theory of algorithmic complexity for problems. This theory is aimed at  ',
  'determination of computer abilities in solving different problems and estimation of  ',
  'resources that computers need to do this. Here we build the part of this theory related to  ',
  'static measures of algorithms. At first, we consider problems for finite words and study  ',
  'algorithmic complexity of such problems, building optimal complexity measures. Then we  ',
  'consider problems for such infinit

# Homework 4

In [153]:
def find_best_match(sentence):
    # Preprocess the input sentence
    processed_query = preprocess([sentence])[0]

    # TF-IDF vector
    query_vector_tfidf = vectorizer.transform([processed_query])
    similarities_tfidf = cosine_similarity(query_vector_tfidf, tfidf_matrix)
    best_match_tfidf_index = similarities_tfidf.argmax()
    best_match_tfidf_score = similarities_tfidf[0, best_match_tfidf_index]
    best_match_tfidf_paragraph = paragraphs[best_match_tfidf_index]

    # Reduced SVD vector
    query_vector_svd = svd.transform(query_vector_tfidf)
    similarities_svd = cosine_similarity(query_vector_svd, reduced_matrix)
    best_match_svd_index = similarities_svd.argmax()
    best_match_svd_score = similarities_svd[0, best_match_svd_index]
    best_match_svd_paragraph = paragraphs[best_match_svd_index]

    # Doc2Vec model
    query_vector_doc2vec = model.infer_vector([processed_query])
    most_similar_doc = model.dv.most_similar(positive=[query_vector_doc2vec])[0]
    best_match_doc2vec_index = most_similar_doc[0]
    best_match_doc2vec_score = most_similar_doc[1]
    best_match_doc2vec_paragraph = paragraphs[best_match_doc2vec_index]

    # Create and return the result dictionary
    result_dict = {
        "TF-IDF Best Match": {
            "Index": best_match_tfidf_index,
            "Score": best_match_tfidf_score,
            "Paragraph": best_match_tfidf_paragraph
        },
        "SVD Best Match": {
            "Index": best_match_svd_index,
            "Score": best_match_svd_score,
            "Paragraph": best_match_svd_paragraph
        },
        "Doc2Vec Best Match": {
            "Index": best_match_doc2vec_index,
            "Score": best_match_doc2vec_score,
            "Paragraph": best_match_doc2vec_paragraph
        }
    }

    return result_dict

# Example usage
input_sentence = 'love in the garden'
result = find_best_match(input_sentence)
print(result)


{'TF-IDF Best Match': {'Index': 206, 'Score': 0.49590564429701683, 'Paragraph': {'paragraph': ' His flower had said that she was one of a kind in all the universe. And here were five thousand of them, all alike, in a single garden! ‘She would be very upset,’ he said to himself, ‘if she came to this garden.', 'nr': 212, 'bookID': 1}}, 'SVD Best Match': {'Index': 206, 'Score': 0.6588293625716577, 'Paragraph': {'paragraph': ' His flower had said that she was one of a kind in all the universe. And here were five thousand of them, all alike, in a single garden! ‘She would be very upset,’ he said to himself, ‘if she came to this garden.', 'nr': 212, 'bookID': 1}}, 'Doc2Vec Best Match': {'Index': 77, 'Score': 0.10281216353178024, 'Paragraph': {'paragraph': 'CHAPTER 8 I soon learned more about the flower. Flowers were simple things on the little prince’s planet and took up very little room. They only had a single ring of petals and were not at all demanding.', 'nr': 80, 'bookID': 1}}}


# Homework 5

In [158]:
def find_k_best_matches_tfidf(sentence, k):
    # Preprocess the input sentence
    processed_query = preprocess([sentence])[0]

    # Vectorize the query using the TF-IDF vectorizer
    query_vector = vectorizer.transform([processed_query])

    # Calculate cosine similarities with all paragraphs
    similarities = cosine_similarity(query_vector, tfidf_matrix)

    # Find K best matches
    best_matches = []
    for _ in range(k):
        nearest_neighbor_index = similarities.argmax()
        similarity_score = similarities[0, nearest_neighbor_index]
        most_similar_paragraph = paragraphs[nearest_neighbor_index]

        # Append information to the result list
        best_matches.append({
            "Index": nearest_neighbor_index,
            "Score": similarity_score,
            "Paragraph": most_similar_paragraph
        })

        # Set the similarity to zero for the already selected paragraph
        similarities[0, nearest_neighbor_index] = 0

    # Sort the result list by similarity score in descending order
    best_matches = sorted(best_matches, key=lambda x: x["Score"], reverse=True)

    # Display results on separate lines
    for match in best_matches:
        print(f"Index: {match['Index']}, Score: {match['Score']}")
        print(match['Paragraph'])
        print("\n" + "-"*50 + "\n")

    return best_matches

# Example usage
input_sentence = 'love is a tree in the eden garden'
k = 10
result_tfidf = find_k_best_matches_tfidf(input_sentence, k)


Index: 206, Score: 0.39074204518466904
{'paragraph': ' His flower had said that she was one of a kind in all the universe. And here were five thousand of them, all alike, in a single garden! ‘She would be very upset,’ he said to himself, ‘if she came to this garden.', 'nr': 212, 'bookID': 1}

--------------------------------------------------

Index: 303, Score: 0.22446290839755653
{'paragraph': ' He remained unmoving for an second. He didn’t make any sound. He dropped gently like a tree. There was not even a thud as he fell on the sand..', 'nr': 309, 'bookID': 1}

--------------------------------------------------

Index: 265, Score: 0.20917507629107732
{'paragraph': ' It felt warm, like a present. When I was a little boy, the lights on the Christmas tree, the music of Midnight Mass, the tender smiling faces, made up for the eagerness of receiving gifts on the morning of Christmas.', 'nr': 271, 'bookID': 1}

--------------------------------------------------

Index: 91, Score: 0.20800

## HTML Captions

## Generated Captions

### !!! do not run locally !!!

In [148]:
!pip install transformers
!pip install accelerate

In [ ]:
from transformers import FuyuForCausalLM, FuyuProcessor

model_id = "adept/fuyu-8b"
processor = FuyuProcessor.from_pretrained(model_id)
model = FuyuForCausalLM.from_pretrained(model_id, device_map="cuda:0")


In [ ]:
import requests
import torch
from PIL import Image
from transformers import AutoTokenizer, FuyuForCausalLM, FuyuImageProcessor, FuyuProcessor

device = "mps"

# Metal supports `bfloat16` in Sonoma, but it still doesn't work
dtype = torch.bfloat16 if device != "mps" else torch.float16

model_id = "adept/fuyu-8b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = FuyuForCausalLM.from_pretrained(model_id, device_map=device, torch_dtype=dtype)
processor = FuyuProcessor(image_processor=FuyuImageProcessor(), tokenizer=tokenizer)

In [ ]:
prompt = "Generate a coco-style caption.\n"

In [ ]:
generation_text

In [ ]:
image = Image.open('Koolhaas__Elements_of_Architecture-2_Images/page_10_image_1.png')

In [ ]:
inputs = processor(text=prompt, images=image, return_tensors="pt").to('mps')


# autoregressively generate text
generation_output = model.generate(**inputs, max_new_tokens=7)
generation_text = processor.batch_decode(generation_output[:, -7:], skip_special_tokens=True)

In [ ]:
generation_text